<a href="https://colab.research.google.com/github/Aliht72/Data-mining-project/blob/main/Classifaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u312-b07-0ubuntu1~18.04 [28.2 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jdk-headless

In [ ]:
!wget https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz

--2022-05-10 04:09:33--  https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272637746 (260M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop2.7.tgz’

spark-3.2.1-bin-had 100%[===================>] 260.01M  72.8MB/s    in 4.5s    

2022-05-10 04:09:38 (57.8 MB/s) - ‘spark-3.2.1-bin-hadoop2.7.tgz’ saved [272637746/272637746]



In [ ]:
!tar xf spark-3.2.1-bin-hadoop2.7.tgz


In [ ]:
!pip install -q findspark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
findspark.find()

'/content/spark-3.2.1-bin-hadoop2.7'

In [ ]:
import pyspark
import numpy as np
import pandas as pd

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('flamingo').getOrCreate()

df = spark.read.csv('./combined-data.csv', sep=',', header=True, inferSchema=True, nullValue='NA')


In [ ]:
removeAllNA = df.na.drop()

In [ ]:
from pyspark.sql import functions as F
categories = removeAllNA.select("platformType").distinct().rdd.flatMap(lambda x: x).collect()

exprs = [F.when(F.col("platformType") == category, 1).otherwise(0).alias(category)
         for category in categories]

AfterDummy = removeAllNA.select("count_buyId", "avg_price" , "count_hits" , "count_gameclicks" , "teamLevel" ,  *exprs)


In [ ]:
AfterDummy.show(5)

+-----------+---------+----------+----------------+---------+------+-------+-----+---+-------+
|count_buyId|avg_price|count_hits|count_gameclicks|teamLevel|iphone|android|linux|mac|windows|
+-----------+---------+----------+----------------+---------+------+-------+-----+---+-------+
|          1|      1.0|         0|              39|        1|     0|      1|    0|  0|      0|
|          1|     10.0|         9|             129|        1|     1|      0|    0|  0|      0|
|          1|      5.0|        14|             102|        1|     0|      1|    0|  0|      0|
|          1|      3.0|         4|              39|        1|     0|      1|    0|  0|      0|
|          1|      3.0|        10|              90|        1|     0|      1|    0|  0|      0|
+-----------+---------+----------+----------------+---------+------+-------+-----+---+-------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import when, col

AfterDummy = AfterDummy.withColumn('Role', when(col('avg_price') > 5 , 'Main Role').otherwise('Others'))
AfterDummy.show()

+-----------+---------+----------+----------------+---------+------+-------+-----+---+-------+---------+
|count_buyId|avg_price|count_hits|count_gameclicks|teamLevel|iphone|android|linux|mac|windows|     Role|
+-----------+---------+----------+----------------+---------+------+-------+-----+---+-------+---------+
|          1|      1.0|         0|              39|        1|     0|      1|    0|  0|      0|   Others|
|          1|     10.0|         9|             129|        1|     1|      0|    0|  0|      0|Main Role|
|          1|      5.0|        14|             102|        1|     0|      1|    0|  0|      0|   Others|
|          1|      3.0|         4|              39|        1|     0|      1|    0|  0|      0|   Others|
|          1|      3.0|        10|              90|        1|     0|      1|    0|  0|      0|   Others|
|          1|     20.0|         8|              51|        1|     1|      0|    0|  0|      0|Main Role|
|          2|      2.5|         6|              51|    

In [ ]:
Role_binary= AfterDummy.withColumn('label', (AfterDummy.Role == "Others").cast('integer'))


In [ ]:
Role_binary.drop('avg_price')

DataFrame[count_buyId: int, count_hits: int, count_gameclicks: int, teamLevel: int, iphone: int, android: int, linux: int, mac: int, windows: int, Role: string, label: int]

In [ ]:
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=[
    'count_buyId', 'count_hits',
    'count_gameclicks', 
    'teamLevel',
    'iphone', 'android', 'linux' , 'mac' , 'windows'
], outputCol='features') 

# Consolidate predictor columns
flamingo_assembled = assembler.transform(Role_binary)

# Check the resulting column
flamingo_assembled.select('features', 'label').show(5, truncate=False)

+----------------------------------------+-----+
|features                                |label|
+----------------------------------------+-----+
|(9,[0,2,3,5],[1.0,39.0,1.0,1.0])        |1    |
|[1.0,9.0,129.0,1.0,1.0,0.0,0.0,0.0,0.0] |0    |
|[1.0,14.0,102.0,1.0,0.0,1.0,0.0,0.0,0.0]|1    |
|[1.0,4.0,39.0,1.0,0.0,1.0,0.0,0.0,0.0]  |1    |
|[1.0,10.0,90.0,1.0,0.0,1.0,0.0,0.0,0.0] |1    |
+----------------------------------------+-----+
only showing top 5 rows



In [ ]:
# Split into training and test sets in a 80:20 ratio
flamingo_train, flamingo_test= flamingo_assembled.randomSplit([0.8, 0.2], seed=17)

# Check that training set has around 80% of records
training_ratio = flamingo_train.count() / flamingo_assembled.count()
print(training_ratio)

0.8015591778880227


In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(flamingo_test)

# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(flamingo_test)
prediction.select('label', 'prediction', 'probability').show(5, False)


+-----+----------+-----------------------------------------+
|label|prediction|probability                              |
+-----+----------+-----------------------------------------+
|1    |1.0       |[0.014285714285714285,0.9857142857142858]|
|1    |1.0       |[0.014285714285714285,0.9857142857142858]|
|1    |1.0       |[0.014285714285714285,0.9857142857142858]|
|1    |1.0       |[0.014285714285714285,0.9857142857142858]|
|1    |1.0       |[0.014285714285714285,0.9857142857142858]|
+-----+----------+-----------------------------------------+
only showing top 5 rows



In [ ]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label = 1').count()
FP = prediction.filter('prediction = 1 AND label = 0').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|    7|
|    0|       0.0|   84|
|    1|       1.0|  167|
|    0|       1.0|   22|
+-----+----------+-----+

0.8964285714285715
